In [1]:
# imports
from pydataset import data
import wrangle as wr

import numpy as np
import pandas as pd
from sklearn.feature_selection import SelectKBest, RFE, f_regression, SequentialFeatureSelector

Load in successful, awaiting commands...


In [2]:
tips = data('tips')

In [3]:
tips.head()

,total_bill,tip,sex,smoker,day,time,size
1,16.99,1.01,Female,No,Sun,Dinner,2
2,10.34,1.66,Male,No,Sun,Dinner,3
3,21.01,3.50,Male,No,Sun,Dinner,3
4,23.68,3.31,Male,No,Sun,Dinner,2
5,24.59,3.61,Female,No,Sun,Dinner,4


In [4]:
tips.shape

(244, 7)

### Create a column named price_per_person. This should be the total bill divided by the party size

In [5]:
# checking data types
tips.dtypes

total_bill    float64
tip           float64
sex            object
smoker         object
day            object
time           object
size            int64
dtype: object

In [6]:
# creating the column
tips['price_per_person'] = round((tips.total_bill) / tips['size'],2)
tips

,total_bill,tip,sex,smoker,day,time,size,price_per_person
1,16.99,1.01,Female,No,Sun,Dinner,2,8.49
2,10.34,1.66,Male,No,Sun,Dinner,3,3.45
3,21.01,3.50,Male,No,Sun,Dinner,3,7.00
4,23.68,3.31,Male,No,Sun,Dinner,2,11.84
5,24.59,3.61,Female,No,Sun,Dinner,4,6.15
...,...,...,...,...,...,...,...,...
240,29.03,5.92,Male,No,Sat,Dinner,3,9.68
241,27.18,2.00,Female,Yes,Sat,Dinner,2,13.59
242,22.67,2.00,Male,Yes,Sat,Dinner,2,11.34
243,17.82,1.75,Male,No,Sat,Dinner,2,8.91


### Before using any of the methods discussed in the lesson, which features do you think would be most important for predicting the tip amount?

* total bill and size

In [29]:
from sklearn.model_selection import train_test_split

In [43]:
# split and scale the data
train, test = train_test_split(tips, test_size = .2, random_state=123)
train, validate = train_test_split(train, test_size = .25, random_state=123)

In [44]:
# verify split size
train.shape, validate.shape, test.shape

((146, 8), (49, 8), (49, 8))

In [47]:
train.dtypes

total_bill          float64
tip                 float64
sex                  object
smoker               object
day                  object
time                 object
size                  int64
price_per_person    float64
dtype: object

In [49]:
# encode categoricals off the train, validate, test and make new encoded_df
dummy_df1 = pd.get_dummies(train[['sex','smoker','day','time']],\
                           dummy_na=False, drop_first=[True, True, True, True])
dummy_df2 = pd.get_dummies(validate[['sex','smoker','day','time']],\
                           dummy_na=False, drop_first=[True, True, True, True])
dummy_df3 = pd.get_dummies(test[['sex','smoker','day','time']],\
                           dummy_na=False, drop_first=[True, True, True, True])
dummy_df1.head()

,sex_Male,smoker_Yes,day_Sat,day_Sun,day_Thur,time_Lunch
225,1,1,0,0,0,1
182,1,1,0,1,0,0
103,0,1,1,0,0,0
165,0,1,0,1,0,0
74,0,1,1,0,0,0


In [58]:
# concat back to encoded_train, validate, test and drop object columns
encoded_train = pd.concat([train.drop(columns=['sex','smoker','day','time']), dummy_df1], axis = 1)
encoded_validate = pd.concat([validate.drop(columns=['sex','smoker','day','time']), dummy_df2], axis = 1)
encoded_test = pd.concat([test.drop(columns=['sex','smoker','day','time']), dummy_df3], axis = 1)

In [59]:
encoded_train.head()

,total_bill,tip,size,price_per_person,sex_Male,smoker_Yes,day_Sat,day_Sun,day_Thur,time_Lunch
225,13.42,1.58,2,6.71,1,1,0,0,0,1
182,23.33,5.65,2,11.66,1,1,0,1,0,0
103,44.30,2.50,3,14.77,0,1,1,0,0,0
165,17.51,3.00,2,8.76,0,1,0,1,0,0
74,25.28,5.00,2,12.64,0,1,1,0,0,0


In [91]:
train.head()

,total_bill,tip,sex,smoker,day,time,size,price_per_person
225,13.42,1.58,Male,Yes,Fri,Lunch,2,6.71
182,23.33,5.65,Male,Yes,Sun,Dinner,2,11.66
103,44.30,2.50,Female,Yes,Sat,Dinner,3,14.77
165,17.51,3.00,Female,Yes,Sun,Dinner,2,8.76
74,25.28,5.00,Female,Yes,Sat,Dinner,2,12.64


In [62]:
# create model variables from encoded dfs
X_train, y_train = encoded_train.drop(columns=['tip']), encoded_train.tip
X_validate, y_validate = encoded_validate.drop(columns=['tip']), encoded_validate.tip
X_test, y_test = encoded_test.drop(columns=['tip']), encoded_test.tip

In [61]:
X_train.head()

,total_bill,size,price_per_person,sex_Male,smoker_Yes,day_Sat,day_Sun,day_Thur,time_Lunch
225,13.42,2,6.71,1,1,0,0,0,1
182,23.33,2,11.66,1,1,0,1,0,0
103,44.30,3,14.77,0,1,1,0,0,0
165,17.51,2,8.76,0,1,0,1,0,0
74,25.28,2,12.64,0,1,1,0,0,0


#### Use select k best to select the top 2 features for predicting tip amount. What are they?

In [80]:
# MAKE the thing
kbest = SelectKBest(f_regression, k=2)

# FIT the thing
kbest.fit(X_train, y_train)

SelectKBest(k=2, score_func=<function f_regression at 0x7fe1102c94c0>)

In [84]:
# statistical f-value / feature's scores & p-values:
kbest.scores_, kbest.pvalues_

(array([1.11115028e+02, 5.76071336e+01, 1.38356417e+01, 5.96049171e-01,
        8.45071808e-02, 2.50128222e-02, 3.28560161e+00, 9.78681779e-01,
        2.18955757e+00]),
 array([1.30561953e-19, 3.66901154e-12, 2.85187703e-04, 4.41354619e-01,
        7.71698118e-01, 8.74556685e-01, 7.19730319e-02, 3.24183852e-01,
        1.41133637e-01]))

In [86]:
# what was sent in
kbest.feature_names_in_

array(['total_bill', 'size', 'price_per_person', 'sex_Male', 'smoker_Yes',
       'day_Sat', 'day_Sun', 'day_Thur', 'time_Lunch'], dtype=object)

In [87]:
kbest_results = pd.DataFrame(dict(p=kbest.pvalues_, f=kbest.scores_),\
                             index = X_train.columns)
kbest_results.head()

,p,f
total_bill,1.305620e-19,111.115028
size,3.669012e-12,57.607134
price_per_person,2.851877e-04,13.835642
sex_Male,4.413546e-01,0.596049
smoker_Yes,7.716981e-01,0.084507


In [88]:
# uh, what?
kbest.get_support()

array([ True,  True, False, False, False, False, False, False, False])

In [89]:
# human readable
X_train_KBtransformed = pd.DataFrame(kbest.transform(X_train),\
                                     columns = X_train.columns[kbest.get_support()],\
                                     index=X_train.index)
X_train_KBtransformed.head()

,total_bill,size
225,13.42,2.0
182,23.33,2.0
103,44.30,3.0
165,17.51,2.0
74,25.28,2.0


### Takeaway:

I was right! Total bill and size are the top two features for KBest.

#### Use recursive feature elimination to select the top 2 features for tip amount. What are they?


In [64]:
from sklearn.linear_model import LinearRegression

In [65]:
lr = LinearRegression()

In [73]:
# make the thing
rfe = RFE(lr, n_features_to_select=2)

# fit the thing
rfe.fit(X_train, y_train)

RFE(estimator=LinearRegression(), n_features_to_select=2)

In [74]:
# Get feature ranking
# Selected features are assigned a rank 1

rfe.ranking_

array([4, 8, 5, 3, 6, 2, 7, 1, 1])

In [75]:
# human readable form
pd.DataFrame({'rfe_ranking':rfe.ranking_},index=X_train.columns)

,rfe_ranking
total_bill,4
size,8
price_per_person,5
sex_Male,3
smoker_Yes,6
day_Sat,2
day_Sun,7
day_Thur,1
time_Lunch,1


In [76]:
rfe.get_support()

array([False, False, False, False, False, False, False,  True,  True])

In [77]:
# human readable
X_train_RFEtransformed = pd.DataFrame(rfe.transform(X_train),index=X_train.index,
    columns = X_train.columns[rfe.support_])

In [79]:
X_train_RFEtransformed.head()

,day_Thur,time_Lunch
225,0.0,1.0
182,0.0,0.0
103,0.0,0.0
165,0.0,0.0
74,0.0,0.0


### Takeaway:

Interesting...? I don't really know what this means

#### Why do you think select k best and recursive feature elimination might give different answers for the top features? Does this change as you change the number of features you are selecting?

* RFE evaluates on importance to the model. Encoded variables are given more weight?

* KBest isolates each feature against the target based on correlation.

#### Write a function named select_kbest that takes in the predictors (X), the target (y), and the number of features to select (k) and returns the names of the top k selected features based on the SelectKBest class. Test your function with the tips dataset. You should see the same results as when you did the process manually.

In [99]:
def select_kbest(X_train, y_train, n_features):
    # MAKE the thing
    kbest = SelectKBest(f_regression, k=n_features)

    # FIT the thing
    kbest.fit(X_train, y_train)
    
    # USE the thing and put into DF
    X_train_KBtransformed = pd.DataFrame(kbest.transform(X_train),\
                                     columns = X_train.columns[kbest.get_support()],\
                                     index=X_train.index)
    
    # return column names
    print(f"These are the top {n_features} columns selected from Select K Best model:")
    return X_train_KBtransformed.columns.tolist()

In [100]:
select_kbest(X_train, y_train, 2)

These are the top 2 columns selected from Select K Best model:


['total_bill', 'size']

#### Write a function named rfe that takes in the predictors, the target, and the number of features to select. It should return the top k features based on the RFE class. Test your function with the tips dataset. You should see the same results as when you did the process manually.



In [101]:
def rfe(X_train, y_train, n_features):
    # make the thing
    rfe = RFE(LinearRegression(), n_features_to_select = n_features)

    # fit the thing
    rfe.fit(X_train, y_train)
    
    # use the thing
    X_train_RFEtransformed = pd.DataFrame(rfe.transform(X_train),index=X_train.index,
    columns = X_train.columns[rfe.support_])
    
     # return column names
    print(f"These are the top {n_features} columns selected from RFE model:")
    return X_train_RFEtransformed.columns.tolist()

In [102]:
rfe(X_train, y_train, 2)

These are the top 2 columns selected from RFE model:


['day_Thur', 'time_Lunch']

#### Load the swiss dataset and use all the other features to predict Fertility. Find the top 3 features using both select k best and recursive feature elimination (use the functions you just built to help you out).

In [103]:
swiss = data('swiss')

In [104]:
swiss.head()

,Fertility,Agriculture,Examination,Education,Catholic,Infant.Mortality
Courtelary,80.2,17.0,15,12,9.96,22.2
Delemont,83.1,45.1,6,9,84.84,22.2
Franches-Mnt,92.5,39.7,5,5,93.40,20.2
Moutier,85.8,36.5,12,7,33.77,20.3
Neuveville,76.9,43.5,17,15,5.16,20.6


In [106]:
swiss.dtypes

Fertility           float64
Agriculture         float64
Examination           int64
Education             int64
Catholic            float64
Infant.Mortality    float64
dtype: object

In [109]:
swiss.shape # tiny

(47, 6)

In [112]:
# split and scale the data
train, test = train_test_split(swiss, test_size = .2, random_state=123)
train, validate = train_test_split(train, test_size = .25, random_state=123)

In [115]:
train.shape, validate.shape, test.shape

((27, 6), (10, 6), (10, 6))

In [118]:
train.head()

,Fertility,Agriculture,Examination,Education,Catholic,Infant.Mortality
Rive Droite,44.7,46.6,16,29,50.43,18.2
Aubonne,66.9,67.5,14,7,2.27,19.1
Rolle,60.5,60.8,16,10,7.72,16.3
Lavaux,65.1,73.0,19,9,2.84,20.0
Nyone,56.6,50.9,22,12,15.14,16.7


In [120]:
# create model variables from dfs
X_train, y_train = train.drop(columns=['Fertility']), train.Fertility
X_validate, y_validate = validate.drop(columns=['Fertility']), validate.Fertility
X_test, y_test = test.drop(columns=['Fertility']), test.Fertility

In [121]:
select_kbest(X_train, y_train, 3)

These are the top 2 columns selected from Select K Best model:


['Examination', 'Catholic', 'Infant.Mortality']

In [122]:
rfe(X_train, y_train, 3)

These are the top 3 columns selected from RFE model:


['Examination', 'Education', 'Infant.Mortality']